In [ ]:
import pandas as pd
from google.colab import files
from io import BytesIO

# Helper: Auto-detect column
def auto_detect_column(columns, keywords):
    for kw in keywords:
        for col in columns:
            if isinstance(col, str) and kw.lower() in col.lower():
                return col
    return None

# Helper: Normalize names
def normalize_name(name):
    if pd.isna(name):
        return ""
    return str(name).lower().strip()

# Helper: Partial match
def partial_match(n1, n2):
    return bool(set(n1.split()) & set(n2.split()))

# STEP 1: Upload File 1
print("Upload File 1 (source data with name + hire date):")
uploaded1 = files.upload()
file1_name = next(iter(uploaded1))

skip1 = int(input("How many rows to skip in File 1 (e.g. headers/notes)? "))
df1_raw = pd.read_excel(BytesIO(uploaded1[file1_name]), skiprows=skip1)

# STEP 2: Upload File 2
print("Upload File 2 (to fill Date of Hire):")
uploaded2 = files.upload()
file2_name = next(iter(uploaded2))

sheet_names = pd.ExcelFile(BytesIO(uploaded2[file2_name])).sheet_names
print(f"Available sheets in File 2: {sheet_names}")
sheet2_name = input("Enter the sheet name to process in File 2: ")

skip2 = int(input("How many rows to skip in File 2 (e.g. headers/notes)? "))
df2_raw = pd.read_excel(BytesIO(uploaded2[file2_name]), sheet_name=sheet2_name, skiprows=skip2)

# Copy raw DataFrames for editing (to preserve originals)
df1 = df1_raw.copy()
df2_edit = df2_raw.copy()

# ==== AUTO DETECT COLUMNS IN FILE 1 ====
name_col1 = auto_detect_column(df1.columns, ["Driver Name", "Driver Full Name", "Driver Last Name", "Name"])
date_col1 = auto_detect_column(df1.columns, ["Date of Hire", "Hire Date", "DOH"])
cdl_col1 = auto_detect_column(df1.columns, ["CDL", "CDL Number", "CDL No", "DL No"])

if name_col1 is None:
    name_col1 = input("File 1: Enter column name for Driver Name: ")
if date_col1 is None:
    date_col1 = input("File 1: Enter column name for Date of Hire: ")

# ==== AUTO DETECT COLUMNS IN FILE 2 ====
name_col2 = auto_detect_column(df2_edit.columns, ["Driver Name", "Driver Full Name", "Name of Driver"])
date_col2 = auto_detect_column(df2_edit.columns, ["Date of Hire", "Hire Date", "DOH"])

if name_col2 is None:
    name_col2 = input("File 2: Enter column name for Driver Name: ")

# If no date column in File 2, create one
if date_col2 is None:
    date_col2 = "Date of Hire"
    df2_edit[date_col2] = pd.NaT

cdl_col2 = auto_detect_column(df2_edit.columns, ["CDL", "CDL Number", "CDL No", "DL No"])

# ==== Normalize names ====
df1['__name1'] = df1[name_col1].apply(normalize_name)
df2_edit['__name2'] = df2_edit[name_col2].apply(normalize_name)

# Ensure dates are datetime
df1[date_col1] = pd.to_datetime(df1[date_col1], errors='coerce')
df2_edit[date_col2] = pd.to_datetime(df2_edit[date_col2], errors='coerce')

# ==== Matching Logic ====
for idx, row in df2_edit[df2_edit[date_col2].isna()].iterrows():
    name2 = row['__name2']
    matched = False

    for _, r1 in df1.iterrows():
        if partial_match(name2, r1['__name1']):
            df2_edit.at[idx, date_col2] = r1[date_col1]
            matched = True
            break

    # Try matching by CDL if name not matched
    if not matched and cdl_col1 and cdl_col2:
        cdl_val = row.get(cdl_col2)
        if pd.notna(cdl_val):
            match = df1[df1[cdl_col1] == cdl_val]
            if not match.empty:
                df2_edit.at[idx, date_col2] = match.iloc[0][date_col1]

# Format the date as MM/DD/YYYY
df2_edit[date_col2] = pd.to_datetime(df2_edit[date_col2], errors='coerce').dt.strftime('%m/%d/%Y')

# ==== REBUILD OUTPUT FILE WITH NO DATA LOSS ====
with pd.ExcelWriter(f"output_{file2_name}", engine='openpyxl') as writer:
    # Write all original sheets from File 2 untouched
    xls = pd.ExcelFile(BytesIO(uploaded2[file2_name]))
    for sheet in xls.sheet_names:
        df_sheet = pd.read_excel(BytesIO(uploaded2[file2_name]), sheet_name=sheet)
        if sheet == sheet2_name:
            # Replace the edited one
            df2_final = pd.concat([df2_raw.iloc[:0], df2_edit], ignore_index=True)
            df2_final.to_excel(writer, sheet_name=sheet, index=False)
        else:
            df_sheet.to_excel(writer, sheet_name=sheet, index=False)

# Download
files.download(f"output_{file2_name}")


Upload File 1 (source data with name + hire date):


Saving Driver Schedule.xlsx to Driver Schedule (14).xlsx
How many rows to skip in File 1 (e.g. headers/notes)? 5
Upload File 2 (to fill Date of Hire):


Saving mvr_Mega Delivery System & Transportation LLC DBA MDS & Transportation --__ Driver Analysis request.xlsx to mvr_Mega Delivery System & Transportation LLC DBA MDS & Transportation --__ Driver Analysis request.xlsx
Available sheets in File 2: ['MVR', 'All Trans']
Enter the sheet name to process in File 2: All Trans
How many rows to skip in File 2 (e.g. headers/notes)? 3


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>